In [ ]:
import tensorflow as tf
print("tensorflow: ", tf.__version__)

from tensorflow import keras
print("keras:", keras.__version__)

In [ ]:
#-- NumPy is a Python library used for working with arrays.
import numpy as np 
#-- Pandas is a Python library used for working with data sets.
import pandas as pd 
#-- Pyplot contains various functions that help matplotlib behave like MATLAB.
import matplotlib
#-- Agg, is a non-interactive backend that can only write to files. 
#-- It is used on Linux, if Matplotlib cannot connect to either an X display.
matplotlib.use('agg')
import matplotlib.pyplot as plt 

#-- OpenCV-Python is a library of Python bindings designed to solve computer vision problems
import cv2
#-- Performs interpolation to up-size or down-size images.
from skimage.transform import resize 

#-- The OS module in python provides functions for interacting with the operating system.
import os
#-- pseudo-random generator
import random 
#-- regular expression
import re

#-- Switching from one backend to another.
from tensorflow.keras import backend as K 
#-- A collection of examples for learning mathematical modelling
from tensorflow.keras import optimizers 
#-- Keras ImageDataGenerator is used for getting the input of the original data and further, 
#--  it makes the transformation of this data on a random basis and gives the output
# from tensorflow.keras.preprocessing.image import ImageDataGenerator 
#-- collection of utilities for array and list manipulation
from tensorflow.keras.utils import to_categorical
#-- Sequential groups a linear stack of layers into a tf.keras.Model.  
#-- Sequential provides training and inference features on this model.
from tensorflow.keras.models import Sequential, load_model 
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import plot_model

In [ ]:
train_dir = "../input/i-can-write/handwritten-numeric-v3/train/"
train_data = []

for i in os.listdir(train_dir):
    sub_directory = os.path.join(train_dir,i)
    count = 0
    for j in os.listdir(sub_directory):
        count+=1
        if count > 3000:
            break
        #-- Load the image, force it to be in grayscale format, and force the size to be 32×32 pixels
        filename = os.path.join(sub_directory,j)
        #read image as grayscale
        img_gray = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        # define a threshold, 128 is the middle of black and white in grayscale
        thresh = 128
        # threshold the image
        img_binary = cv2.threshold(img_gray, thresh, 255, cv2.THRESH_BINARY)[1]
        img = np.array(img_binary)
        img = img.reshape(32,32,1)
        train_data.append([img,i])
len(train_data)

In [ ]:
val_dir = "../input/i-can-write/handwritten-numeric-v3/validation/"
val_data = []

for i in os.listdir(val_dir):
    sub_directory = os.path.join(val_dir,i)
    count = 0
    for j in os.listdir(sub_directory):
        count+=1
        if count > 750:
            break
        #-- Load the image, force it to be in grayscale format, and force the size to be 32×32 pixels
        filename = os.path.join(sub_directory,j)
        #read image as gray scale
        img_gray = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        # define a threshold, 128 is the middle of black and white in grayscale
        thresh = 128
        # threshold the image
        img_binary = cv2.threshold(img_gray, thresh, 255, cv2.THRESH_BINARY)[1]
        img = np.array(img_binary)
        img = img.reshape(32,32,1)
        val_data.append([img,i])
len(val_data)

In [ ]:
random.shuffle(train_data)
random.shuffle(val_data)

train_X = []
_train_Y = []
for features,label in train_data:
    train_X.append(features)
    _train_Y.append(label)
    
val_X = []
_val_Y = []
for features,label in val_data:
    val_X.append(features)
    _val_Y.append(label)

In [ ]:
#-- Inspect a few examples in the dataset
fig = plt.figure()
for i in range(9):
  plt.subplot(3,3,i+1)
  plt.tight_layout()
  plt.imshow(train_X[i], cmap='gray', interpolation='none')
  plt.title("Digit: {}".format(_train_Y[i]))
  plt.xticks([])
  plt.yticks([])
fig

In [ ]:
#-- This is an example of the distribution graph of the pixel values.
fig = plt.figure()
j = 0
plt.subplot(2,1,1)
plt.imshow(train_X[j], cmap='gray', interpolation='none')
plt.title("Digit: {}".format(_train_Y[j]))
plt.xticks([])
plt.yticks([])
plt.subplot(2,1,2)
plt.hist(train_X[j].reshape(1024))
plt.title("Pixel Value Distribution")
fig

In [ ]:
#-- Pixel values range from 0 to 255: the background majority close to 0 (black), 
#--   and those close to 255 (white) representing the digit.

train_X = np.array(train_X)
val_X = np.array(val_X)

train_X = train_X.astype('float32')
val_X = val_X.astype('float32')

train_X = train_X.reshape(train_X.shape[0],32,32,1)
val_X = val_X.reshape(val_X.shape[0],32,32,1)

#-- zero-centered data
#train_X -= np.mean(train_X, axis=0)
#val_X -= np.mean(val_X, axis=0)

#-- Normalize the pixel values to lie between 0 and 1
#train_X /= 255
#val_X /= 255

_train_Y = np.array(_train_Y, dtype='uint8')
_val_Y = np.array(_val_Y, dtype='uint8')

#-- Print the final shape ready for training
print("matrix shape (train_X|_train_Y) :", train_X.shape,_train_Y.shape)
print("matrix shape (  val_X|  _val_Y) :", val_X.shape,_val_Y.shape)

In [ ]:
#-- the truth (Y in machine learning lingo) used for training still holds integer values from 0 to 9.
#-- uint8 is an unsigned 8-bit integer that can represent values 0..255
print(np.unique(_val_Y, return_counts=True))

In [ ]:
#-- one-hot encoding using keras' numpy-related utilities
#-- The vector is all zeroes except in the position for the respective category.
#-- For example, a '5' will be represented by [0,0,0,0,0,1,0,0,0,0].
n_classes = 10
print("Shape (_train_Y) before one-hot encoding:", _train_Y.shape)
print("Shape (  _val_Y) before one-hot encoding:", _val_Y.shape)
train_Y = keras.utils.to_categorical(_train_Y, n_classes)
val_Y = keras.utils.to_categorical(_val_Y, n_classes)
print("Shape (train_Y) after  one-hot encoding:", train_Y.shape)
print("Shape (  val_Y) after  one-hot encoding: ", val_Y.shape)

In [ ]:
#-- Building a linear stack of layers with the sequential model
model = Sequential()

model.add(Conv2D(32,(5,5), kernel_initializer='he_uniform', input_shape=(32,32,1)))
model.add(tf.keras.layers.PReLU())
model.add(MaxPooling2D((2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(32,(3,3), kernel_initializer='he_uniform'))
model.add(tf.keras.layers.PReLU())
model.add(Conv2D(64,(3,3), kernel_initializer='he_uniform'))
model.add(tf.keras.layers.PReLU())
model.add(Conv2D(64,(3,3), kernel_initializer='he_uniform'))
model.add(tf.keras.layers.PReLU())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, kernel_initializer='he_uniform'))
model.add(tf.keras.layers.PReLU())

model.add(Dense(10, activation='softmax'))

model.summary()

# plot model architecture
plot_model(model, show_shapes='true', to_file='cnn.png')

In [ ]:
#-- Compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
#opt = SGD(learning_rate=0.01, momentum=0.9)
#model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)

In [ ]:
#-- Training the model and saving metrics in history
history = model.fit(train_X, train_Y,
          batch_size=128, epochs=10,
          verbose=2,
          validation_data=(val_X, val_Y))

In [ ]:
model.save('handwriting.h5')

In [ ]:
#-- Plotting the metrics
fig = plt.figure()

plt.subplot(2,1,1)
x = np.arange(1., 11., 1.0)
plt.plot(x, history.history['accuracy'])
plt.plot(x, history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.xticks(np.arange(0, 11, 1))
plt.xlim(right=11)
plt.legend(['train', 'validation'], loc='lower right')

plt.subplot(2,1,2)
x = np.arange(1., 11., 1.0)
plt.plot(x, history.history['loss'])
plt.plot(x, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.xticks(np.arange(0, 11, 1))
plt.xlim(right=11)
plt.legend(['train', 'validation'], loc='upper right')

#plt.tight_layout()
fig.subplots_adjust(hspace=0.8, wspace=0.8)
fig

In [ ]:
model = load_model('handwriting.h5')
loss_and_metrics = model.evaluate(val_X, val_Y, verbose=2)

print("Validation Loss", loss_and_metrics[0])
print("Validation Accuracy", loss_and_metrics[1])

In [ ]:
#-- Load the model and create predictions on the validation set
model = load_model('handwriting.h5')
predicted_classes = np.argmax(model.predict(val_X), axis=1)

#-- See which we predicted correctly and which not
correct_indices = np.nonzero(predicted_classes == _val_Y)[0]
incorrect_indices = np.nonzero(predicted_classes != _val_Y)[0]
print()
print(len(correct_indices)," classified correctly")
print(len(incorrect_indices)," classified incorrectly")

In [ ]:
#-- Adapt figure size to accomodate 18 subplots
plt.rcParams['figure.figsize'] = (7,14)

figure_evaluation = plt.figure()

#-- Plot 9 correct predictions
#for i, correct in enumerate(correct_indices[:9]):
#    plt.subplot(6,3,i+1)
#    plt.imshow(val_X[correct].reshape(32,32), cmap='gray', interpolation='none')
#    plt.title(
#      "Predicted: {}, Truth: {}".format(predicted_classes[correct],
#                                        _val_Y[correct]))
#    plt.xticks([])
#    plt.yticks([])

#-- Plot 9 incorrect predictions
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(6,3,i+10)
    plt.imshow(val_X[incorrect].reshape(32,32), cmap='gray', interpolation='none')
    plt.title(
      "Predicted: {}, Truth: {}".format(predicted_classes[incorrect],
                                        _val_Y[incorrect]))
    plt.xticks([])
    plt.yticks([])

figure_evaluation

In [ ]:
def prediction_static(filename):
    number = re.search(r"\d",filename)
    actual = number.group()
    mypath = os.path.join("../input/i-can-write/handwritten-numeric-v3/holdout_static/",filename)
    #read image as grayscale
    img_gray = cv2.imread(mypath, cv2.IMREAD_GRAYSCALE)
    # define a threshold, 128 is the middle of black and white in grayscale
    thresh = 128
    # threshold the image
    img_binary = cv2.threshold(img_gray, thresh, 255, cv2.THRESH_BINARY)[1]
    img = np.invert(img_binary)
    img = img.astype('float32')
    img_re = resize(img,(32,32,1))
    img_re = img_re.reshape(32,32,1)
    #img_re -= np.mean(img_re, axis=0)
    #img_re /= 255
    probabilities = model.predict(np.array([img_re],))[0,:]
    number_to_class = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    index = np.argsort(probabilities)
    if number_to_class[index[9]] == actual:
     grade = "Good Job!"
    else:
     grade = "Hmmm ... Did I make a wrong guess?"
    predictions = {
      "actual":actual,
      "digit":number_to_class[index[9]],
      "prob" :probabilities[index[9]],
      "comment":grade
    }
    return predictions, img_re

In [ ]:
predictions_static = ()
number = ["0","1","2","3","4","5","6","7","8","9"]
for i in number:
    myfile = i + ".jpg"
    predictions, img_re = prediction_static(myfile)
    print(predictions)

In [ ]:
def crop_square(img, size, interpolation=cv2.INTER_AREA):
    h, w = img.shape[:2]
    min_size = np.amin([h,w])
    # Centralize and crop
    crop_img = img[int(h/2-(min_size/2)):int(h/2+(min_size/2)), int(w/2-(min_size/2)):int(w/2+(min_size/2))]
    resized = cv2.resize(crop_img, (size, size), interpolation=interpolation)

    return resized

In [ ]:
def prediction(filename):
    number = re.search(r"\d",filename)
    actual = number.group()
    mypath = os.path.join("../input/i-can-write/handwritten-numeric-v3/holdout/",filename)
    #read image as grayscale
    img_gray = cv2.imread(mypath, cv2.IMREAD_GRAYSCALE)
    # define a threshold, 128 is the middle of black and white in grayscale
    thresh = 128
    # threshold the image
    img_bin = cv2.threshold(img_gray, thresh, 255, cv2.THRESH_BINARY)[1]
    img_bin = np.invert(img_bin)
    img_bin = img_bin.astype('float32')
    crop_rows = img_bin[~np.all(img_bin==0, axis=1), :]
    cropped_image = crop_rows[:, ~np.all(crop_rows==0, axis=0)]
    top, bottom, left, right = [10]*4
    img = cv2.copyMakeBorder(cropped_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])
    img = crop_square(img, 32, cv2.INTER_AREA)
    img = cv2.threshold(img, thresh, 255, cv2.THRESH_BINARY)[1]
    img_re = img.reshape(32,32,1)
    #img_re -= np.mean(img_re, axis=0)
    #img_re /= 255
    probabilities = model.predict(np.array([img_re],))[0,:]
    number_to_class = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    index = np.argsort(probabilities)
    if number_to_class[index[9]] == actual:
     grade = "Good Job!"
    else:
     grade = "Hmmm ... Did I make a wrong guess?"
    predictions = {
      "actual":actual,
      "digit":number_to_class[index[9]],
      "prob" :probabilities[index[9]],
      "comment":grade
    }
    return predictions, img_gray, img_bin, cropped_image, img, img_re

In [ ]:
predictions = ()
number = ["0","1","2","3","4","5","6","7","8","9"]
#number = ["6"]
for i in number:
    myfile = i + ".jpg"
    predictions, img_gray, img_bin, cropped_image, img, img_re = prediction(myfile)
    print(predictions)

In [ ]:
#fig = plt.figure()
#plt.imshow(img, cmap='gray', interpolation='none')
#fig

In [ ]:
#%matplotlib inline
#import seaborn as sns

#img = cv2.imread("../input/i-can-write/handwritten-numeric-v3/validation/4/30000.jpg", cv2.IMREAD_GRAYSCALE)
#thresh = 128
#img = cv2.threshold(img, thresh, 255, cv2.THRESH_BINARY)[1]
#img = np.array(img)
# with np.printoptions(threshold=np.inf):
#    print(img)

#f, ax = plt.subplots(figsize=(32,32))
#sns.heatmap(img, annot=True, fmt='.1f', square=True, cmap="gray")
#plt.show()

In [ ]:
# plot feature map of first conv layer for given image
#from tensorflow.keras.applications.vgg16 import VGG16
#from tensorflow.keras.applications.vgg16 import preprocess_input
#from tensorflow.keras.preprocessing.image import load_img
#from tensorflow.keras.preprocessing.image import img_to_array
#from tensorflow.keras.models import Model
#from matplotlib import pyplot
#from numpy import expand_dims

# load the model
#model = VGG16()
# redefine model to output right after the first hidden layer
#model = Model(inputs=model.inputs, outputs=model.layers[1].output)
#model.summary()
# load the image with the required shape
#img = load_img('../input/i-can-write/handwritten-numeric-v3/validation/3/30004.jpg', target_size=(224, 224))
# convert the image to an array
#img = img_to_array(img)
# expand dimensions so that it represents a single 'sample'
#img = expand_dims(img, axis=0)
# prepare the image (e.g. scale pixel values for the vgg)
#img = preprocess_input(img)
# get feature map for first hidden layer
#feature_maps = model.predict(img)
# plot all 64 maps in an 8x8 squares
#square = 8
#ix = 1
#for _ in range(square):
#    for _ in range(square):
#        # specify subplot and turn of axis
#        ax = pyplot.subplot(square, square, ix)
#        ax.set_xticks([])
#        ax.set_yticks([])
#        # plot filter channel in grayscale
#        pyplot.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
#        ix += 1
# show the figure
#plt.show()

In [ ]:
#import shutil
#shutil.rmtree("/kaggle/working/")